### Logistic Regression with TensorFlow eager API

TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. To follow along with this guide, run the code samples below in an interactive python interpreter.

Logistic Regression can be best understood from here: https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html

#### MNIST handwritten digits  

The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).
![title](../img/mnist.png)

In [1]:
from __future__ import print_function, division, absolute_import
import tensorflow as tf

C:\Users\supratimdas\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# eager api imports
tfe = tf.contrib.eager
tfe.enable_eager_execution()

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot = False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# parameters
learning_rate = 0.1
batch_size = 128
epochs = 1000
display_step = 100

In [5]:
# iterator
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels))
dataset = dataset.repeat().batch(batch_size).prefetch(batch_size)
dataset_Iter = tfe.Iterator(dataset)

In [16]:
# variable declarations
W = tfe.Variable(tf.zeros([784,10]), name='weights')
b = tfe.Variable(tf.zeros([10]), name='biases')

# logistic regression input (Wx + b)
def logistic_regression(inputs):
    return tf.matmul(inputs, W) + b

# inference function can be anything like sigmoid, tanH etc. anything
def loss_function(inference_fn, inputs, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=inference_fn(inputs),labels=labels))

# accuracy function
def accuracy_function(inference_fn, inputs, labels):
    predictions = tf.nn.softmax(inference_fn(inputs))
    correct_preds = tf.equal(tf.argmax(predictions,1), labels)
    return tf.reduce_mean(tf.cast(correct_preds, tf.float32))

In [7]:
# optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
gradient = tfe.implicit_gradients(loss_function)

In [17]:
# training status variables
avg_acc = 0
avg_loss = 0

# training begins 
for i in range(epochs):
    d = dataset_Iter.next()
    x_batch = d[0]
    y_batch = tf.cast(d[1], tf.int64)
    
    # calculating current loss and accuracy
    curr_loss = loss_function(logistic_regression, x_batch, y_batch)
    curr_acc = accuracy_function(logistic_regression, x_batch, y_batch)
    
    # adding up the loss and the accuracy
    avg_acc += curr_acc
    avg_loss += curr_loss
    
    # optimize the loss (adjust the values of W and b in order to minimize the loss)
    optimizer.apply_gradients(gradient(logistic_regression, x_batch, y_batch))
    
    # print losses
    if (i + 1) % display_step == 0 or i == 0:
        if i > 0:
            avg_acc /= display_step
            avg_loss /= display_step
        print("Step:", '%04d' % (i + 1), " loss=", "{:.9f}".format(avg_loss), " accuracy=", "{:.4f}".format(avg_acc))
        average_loss = 0.
        average_acc = 0.

Step: 0001  loss= 2.302585363  accuracy= 0.1094
Step: 0100  loss= 0.978982031  accuracy= 0.7995
Step: 0200  loss= 0.512349308  accuracy= 0.8890
Step: 0300  loss= 0.489089817  accuracy= 0.8807
Step: 0400  loss= 0.450997084  accuracy= 0.8902
Step: 0500  loss= 0.427731633  accuracy= 0.8948
Step: 0600  loss= 0.361523747  accuracy= 0.9133
Step: 0700  loss= 0.395773053  accuracy= 0.9023
Step: 0800  loss= 0.383722723  accuracy= 0.9032
Step: 0900  loss= 0.374225497  accuracy= 0.9069
Step: 1000  loss= 0.338724583  accuracy= 0.9170


In [18]:
# Evaluate model on the test image set
x_test = mnist.test.images
y_test = mnist.test.labels

test_accuracy = accuracy_function(logistic_regression, x_test, y_test)
print("Test Accuracy: {:.4f}".format(test_accuracy))

Test Accuracy: 0.9084
